The following lines preload the data, import necessary libraries and create user defined functions.

In [2]:
import os
import pandas as pd
#import collocation_analysis as ca
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
#from investing_scrape import scrape_headlines
import numpy as np
import re
from decimal import Decimal
import json
import time
import unicodedata
from pandas import DataFrame
from selenium import webdriver
from bs4 import BeautifulSoup

#region FUNCTION DEFINITIONS
def clean_Text(text):
    """
    removes punctuation, stopwords and returns lowercase text in a list of single words
    """    
    print(text)
    # If the string happens to be NaN, the regexp module will throw an error. By checking for the length of the 
    # string I can catch whether it is a string or not for a TypeError will be thrown if it is not a string. 
    try:
        t = len(text)
        #Tokenize the text
        tokenizer = RegexpTokenizer(r'\w+')
        text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
        text = text.lower()
        text = tokenizer.tokenize(text)

        #Clean the text
        clean = [word for word in text if word not in stopwords.words('english')]

        return clean
    
    except TypeError:
        return "null"

# Load the list of positive words
def loadPositive():
    #filepath = os.path.dirname(os.path.abspath(__file__))
    with open('C:\\Users\\afn\\Downloads\\fxnews\\sentiment\\LMC_Positive.csv', 'r') as f:
        pos_lines = f.readlines()
        pos_lex_lists = [word.strip() for word in pos_lines]
        return pos_lex_lists


# Load the list of negative words
def loadNegative():
    #filepath = os.path.dirname(os.path.abspath(__file__))
    with open('C:\\Users\\afn\\Downloads\\fxnews\\sentiment\\LMC_Negative.csv', 'r') as f:
        neg_lines = f.readlines()
        neg_lex_lists = [word.strip() for word in neg_lines]
        return neg_lex_lists


def countNegative(sentence, neg_lex_list, total_headline_count, doc_average_word_count):
    # Sentence is a sentence broken up into a list of words
    # Add a row to the df_frequency dataframe that represents the weight assigned to that word
    # Use the Loughran & McDonald formula to create the weight:
    # Wij = ((1+log(tfij))/(1+log(a)))*log(N/dfi)
    # Where N represents the total number of headlines in the sample
    # dfi the number of headlines containing at least one occurrence of the ith word
    # tfij the raw count of the ith word in the jth document
    # a the average word count in the document
    N = total_headline_count
    headline_score = 0
    a = doc_average_word_count.item()
    print('****Negative****')
    print("N")
    print(N)
    print("A")
    print(a)
    print('*******')
    print(df_frequency)
    print('***THAT WAS FREQUENCY****')
    for word in sentence:
        if word.upper() in neg_lex_list:
            # Find the index of the word then obtain the corresponding frequency and dfi
            wIndex = list(np.where(df_frequency["word"] == word.lower())[0])[0]
            dfi = df_frequency["dfi"][wIndex]
            print(word)
            print('dfi')
            print(dfi)
            # Determine the frequency of this word in this particular headline
            local_freq = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(word), ' '.join(sentence)))
            # Calculate the weight assigned to this word
            weight = ((1+np.log(local_freq))/(1+np.log(a)))*np.log(N/dfi)
            # Add this word score to the headline's score
            headline_score += weight.item()

    return headline_score


def countPositive(sentence, pos_lex_list, total_headline_count, doc_average_word_count):
    # Sentence is a sentence broken up into a list of words
    # Add a row to the df_frequency dataframe that represents the weight assigned to that word
    # Use the Loughran & McDonald formula to create the weight:
    # Wij = ((1+log(tfij))/(1+log(a)))*log(N/dfi)
    # Where N represents the total number of headlines in the sample
    # dfi the number of headlines containing at least one occurrence of the ith word
    # tfij the raw count of the ith word in the jth document
    # a the average word count in the document
    N = total_headline_count
    headline_score = 0
    a = doc_average_word_count.item()
    print('****Negative****')
    print("N")
    print(N)
    print("A")
    print(a)
    print('*******')
    print(df_frequency)
    print('***THAT WAS FREQUENCY****')
    if word.upper() in pos_lex_list:
            # Find the index of the word then obtain the corresponding frequency and dfi
            wIndex = list(np.where(df_frequency["word"] == word.lower())[0])[0]
            dfi = df_frequency["dfi"][wIndex]
            print(word)
            print('dfi')
            print(dfi)
            # Determine the frequency of this word in this particular headline
            local_freq = sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(word), ' '.join(sentence)))
            # Calculate the weight assigned to this word
            weight = ((1+np.log(local_freq))/(1+np.log(a)))*np.log(N/dfi)
            # Add this word score to the headline's score
            headline_score += weight.item()

    return headline_score


def calculateSentiment(clean_headline, positive_list, negative_list, awc):
    # print("CALCULATING SENTIMENT")
    lexicon_negative = countNegative(clean_headline, negative_list, len(df_headlines), awc)
    lexicon_positive = countPositive(clean_headline, positive_list, len(df_headlines), awc)
    return lexicon_positive-lexicon_negative

# Using webdriver to launch the website and allow the javascript to load. Once the JS loads,
# it is possible to extract the source code -> innerHTML
# BeautifulSoup processes the innerHTML, which is then parsed for the desired sections.
# Title, date and url are all stored in a dictionary that has the title as the key.
# The dictionary is stored in a json file for further usage.
def scrape_headlines(starting_page, ending_page, section):
    browser = webdriver.Chrome()
    url_upto_item = "https://www.investing.com/currencies/" + section + '/'
    page_number = starting_page

    # Create a dictionary to store these values. Each title is the key to an entry.
    news = {}
    article_total = 0

    # try:
    #     news[article_total]["source"] = element.span.get_text()[3:]
    # except AttributeError:

    # Loop through all the pages
    while page_number <= ending_page:

        # Join parts of url
        url = url_upto_item + str(page_number)
        # Navigate browser to url
        browser.get(url)

        # Wait 10 seconds for the js to finish loading the webpage
        time.sleep(2)
        # Get innerHTML from webpage
        inner_html = browser.execute_script("return document.body.innerHTML")
        # Parse the innerHTML
        soup = BeautifulSoup(inner_html, 'html.parser')

        # Find all the classes that contain the desired information.
        # Title, url and date are all contained within the 'fxs_floatingMedia..' class in div objects.
        items = soup.find_all("article", class_="articleItem")

        article_counter = 1
        # Clean the gathered text before putting it in the dictionary
        for element in items:
            # Last two articles are not real articles rather some extra stuff on the webpage
            if article_counter < len(items)-2:
                news[article_total] = {}
                news[article_total]["title"] = element.div.a["title"]
                print(element.div.a["title"])
                news[article_total]["date"] = element.findAll("span", {"class": "date"})[0].get_text()[3:]

                # The href standards changed at some point. They are stored in two ways. Here I account for that difference
                if "https://invst.ly" in element.div.a["href"]:
                    news[article_total]["url"] = element.div.a["href"]
                else:
                    news[article_total]["url"] = "https://investing.com" + element.div.a["href"]

                try:
                    news[article_total]["source"] = element.span.span.get_text()[3:]
                except AttributeError:
                    news[article_total]["source"] = element.span.get_text()[3:]

            article_counter += 1
            article_total += 1

        # Increment page number by 1 to navigate to the next page
        page_number += 1

    # ----------------------- DATAFRAME -----------------------

    # create the dataframe from the dictionary
    df = pd.DataFrame.from_dict(news)

    # transpose the columns/rows
    df = df.T

    # Sort by date
    # df = df.sort_values(by=['date'])

    # ----------------------- SAVING -----------------------
    filename = 'headlines_InvestingCom_' + str(starting_page) + '_to_' + str(ending_page) + '.csv'
    # Save the dataframe to a csv file
    df.to_csv(filename, encoding='utf-8')

    print("***********************")
    print("Headline Scraping Done.")
    print("***********************")

    browser.close()

    return df

def wordFrequency(df, content_column):
    # Create a dictionary where to store the words and counts
    frequency = {}

    # Iterate through each row and count the frequency of words
    for index, row in df.iterrows():
        # First clean the text
        text_string = clean_Text(row[content_column])

        # Count word frequency
        for word in text_string:
            if word in frequency:
                frequency[word] += 1
            else:
                frequency[word] = 1

        print(frequency)
    # Create the list with the counts
    frequency_list = frequency.keys()

    # Print words and counts
    for words in frequency_list:
        print(words, frequency[words])

    word_df = pd.DataFrame(list(frequency.items()))
    word_df = word_df.T

    # Save the frequency dictionary to a csv file for further analysis
    with open('word_frequency.csv', 'w') as csv_file:
        word_df.to_csv(csv_file, header=False)
    csv_file.close()

    return word_df

Next, create/import the dataframes containing the data to analyze. In this case the headlines have been downloaded previously, so they are imported from a csv file. The date column is converted to a datetime object for easy manipulation.


In [3]:
df_headlines = pd.read_csv("C:\\Users\\afn\\Downloads\\fxnews\\sentiment\\headline_investing_com_full_sent_1.csv", encoding='iso-8859-1')
df_prices = pd.read_csv("C:\\Users\\afn\\Downloads\\fxnews\\csv_files\\USD_CHF_Sent_Prices.csv")
df_aaii = pd.read_csv("C:\\Users\\afn\\Downloads\\fxnews\\csv_files\\aaii_sentiment.csv")
df_headlines['date'] = pd.to_datetime(df_headlines['date'])
df_prices['date'] = pd.to_datetime(df_prices['date'])
df_aaii['date'] = pd.to_datetime(df_aaii['date'])
print(df_headlines)

     Unnamed: 0       date         source  \
0             0 2018-06-22       FXStreet   
1             1 2018-06-22       FXStreet   
2             2 2018-06-22       FXStreet   
3             3 2018-06-22       FXStreet   
4             4 2018-06-21       FXStreet   
5             5 2018-06-21        Dailyfx   
6             6 2018-06-21  Investing.com   
7             7 2018-06-21       FXStreet   
8             8        NaT            NaN   
9             9        NaT            NaN   
10           10        NaT            NaN   
11           11 2018-06-20       FXStreet   
12           12 2018-06-20        Dailyfx   
13           13 2018-06-20       FXStreet   
14           14 2018-06-20  Investing.com   
15           15 2018-06-19        Dailyfx   
16           16 2018-06-19  Investing.com   
17           17 2018-06-19       FXStreet   
18           18 2018-06-19       FXEmpire   
19           19        NaT            NaN   
20           20        NaT            NaN   
21        

Create lists to store calculations. These lists will eventually become the columns in the final dataframe. The positive and negative lexicon are loaded as well.

In [4]:
pos_lex_list = loadPositive()
neg_lex_list = loadNegative()
# Indicator lists
aaii_sent_list = []
lexicon_sent_list = []
market_sent_list = []
# A list of ngrams that have more weight in the analysis.
# ngram_list = []
ngram_match_list = []
# Where the final aggregate sentiment is stored
final_sent_list = []
final_sent_list_word = []
new_range  = [i * i for i in range(5) if i % 2 == 0]

Determine word frequency in the entire database of headlines. Frequency will be used as a weight to balance 
terms with different levels of popularity.

In [5]:
df_frequency = wordFrequency(df_headlines, "title")


USD/CHF retreats below 0.99 as Trump's comments weigh on market sentiment
{'usd': 1, 'chf': 1, 'retreats': 1, '0': 1, '99': 1, 'trump': 1, 'comments': 1, 'weigh': 1, 'market': 1, 'sentiment': 1}
CHF remains âhighly valuedâ â Danske Bank
{'usd': 1, 'chf': 2, 'retreats': 1, '0': 1, '99': 1, 'trump': 1, 'comments': 1, 'weigh': 1, 'market': 1, 'sentiment': 1, 'remains': 1, 'â': 2, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 1}
USD/CHF Technical Analysis: a bearish double-top formation on H4
{'usd': 2, 'chf': 3, 'retreats': 1, '0': 1, '99': 1, 'trump': 1, 'comments': 1, 'weigh': 1, 'market': 1, 'sentiment': 1, 'remains': 1, 'â': 2, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 1, 'technical': 1, 'analysis': 1, 'bearish': 1, 'double': 1, 'top': 1, 'formation': 1, 'h4': 1}
USD/CHF positive while above 0.9875/72 â Commerzbank
{'usd': 3, 'chf': 4, 'retreats': 1, '0': 2, '99': 1, 'trump': 1, 'comments': 1, 'weigh': 1, 'market': 1, 'sentiment': 1, 'remains': 1, 'â': 3, 'highly': 1, 

{'usd': 21, 'chf': 18, 'retreats': 1, '0': 9, '99': 2, 'trump': 1, 'comments': 1, 'weigh': 1, 'market': 2, 'sentiment': 2, 'remains': 2, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 1, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 5, 'index': 1, 'extends': 1, 'losses': 1, 'safe': 2, 'currencies': 1, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 2, 'hold': 2, 'swiss': 1, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 9, 'u': 9, 'l': 18, 'trims': 1, 'early': 1, 'gains': 1, 'back': 1, 'around': 1, 'mid': 2, '9900s': 1, 'demand': 1, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 1, 'forex': 1, 'edges': 1, 'higher': 1, 'amid': 1, 'lull': 1, 'trade': 1, 'rhetoric': 1, 'price': 1, 'action': 1, 'setups': 2, 'sets': 1, 'fresh':

{'usd': 22, 'chf': 19, 'retreats': 1, '0': 10, '99': 2, 'trump': 1, 'comments': 1, 'weigh': 1, 'market': 2, 'sentiment': 2, 'remains': 2, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 6, 'index': 1, 'extends': 1, 'losses': 1, 'safe': 2, 'currencies': 1, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 2, 'hold': 2, 'swiss': 3, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 12, 'u': 13, 'l': 24, 'trims': 1, 'early': 1, 'gains': 2, 'back': 2, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 1, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 1, 'forex': 5, 'edges': 1, 'higher': 1, 'amid': 1, 'lull': 1, 'trade': 1, 'rhetoric': 1, 'price': 1, 'action': 1, 'setups': 2, 'sets': 1, 'fres

{'usd': 22, 'chf': 19, 'retreats': 1, '0': 10, '99': 2, 'trump': 1, 'comments': 1, 'weigh': 2, 'market': 2, 'sentiment': 2, 'remains': 6, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 29, 'index': 5, 'extends': 2, 'losses': 3, 'safe': 3, 'currencies': 2, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 2, 'hold': 2, 'swiss': 3, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 21, 'u': 32, 'l': 42, 'trims': 2, 'early': 1, 'gains': 5, 'back': 6, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 2, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 1, 'forex': 28, 'edges': 1, 'higher': 2, 'amid': 3, 'lull': 1, 'trade': 5, 'rhetoric': 1, 'price': 1, 'action': 1, 'setups': 2, 'sets': 1, 'fr

Forex - Dollar Hovers Near 3-Year Lows, U.S. Housing Sector Data on Tap
{'usd': 22, 'chf': 19, 'retreats': 1, '0': 10, '99': 2, 'trump': 1, 'comments': 1, 'weigh': 2, 'market': 2, 'sentiment': 2, 'remains': 7, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 34, 'index': 6, 'extends': 2, 'losses': 3, 'safe': 3, 'currencies': 2, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 3, 'hold': 2, 'swiss': 3, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 24, 'u': 38, 'l': 48, 'trims': 2, 'early': 1, 'gains': 5, 'back': 7, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 2, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 1, 'forex': 33, 'edges': 1, 'higher': 3, 'amid': 3, 'lull': 1, 'trade'

{'usd': 22, 'chf': 19, 'retreats': 1, '0': 10, '99': 2, 'trump': 1, 'comments': 2, 'weigh': 2, 'market': 2, 'sentiment': 2, 'remains': 13, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 63, 'index': 13, 'extends': 4, 'losses': 4, 'safe': 3, 'currencies': 4, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 3, 'hold': 2, 'swiss': 4, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 33, 'u': 53, 'l': 66, 'trims': 2, 'early': 1, 'gains': 8, 'back': 7, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 2, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 1, 'forex': 60, 'edges': 2, 'higher': 8, 'amid': 3, 'lull': 1, 'trade': 9, 'rhetoric': 2, 'price': 1, 'action': 1, 'setups': 2, 'sets': 1, '

{'usd': 22, 'chf': 19, 'retreats': 1, '0': 10, '99': 2, 'trump': 1, 'comments': 2, 'weigh': 2, 'market': 2, 'sentiment': 2, 'remains': 14, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 67, 'index': 13, 'extends': 4, 'losses': 4, 'safe': 3, 'currencies': 4, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 3, 'hold': 2, 'swiss': 4, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 36, 'u': 58, 'l': 72, 'trims': 2, 'early': 1, 'gains': 8, 'back': 7, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 2, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 1, 'forex': 64, 'edges': 2, 'higher': 8, 'amid': 3, 'lull': 1, 'trade': 9, 'rhetoric': 2, 'price': 1, 'action': 1, 'setups': 2, 'sets': 1, '

{'usd': 22, 'chf': 19, 'retreats': 2, '0': 10, '99': 2, 'trump': 1, 'comments': 2, 'weigh': 2, 'market': 2, 'sentiment': 2, 'remains': 19, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 97, 'index': 15, 'extends': 4, 'losses': 5, 'safe': 3, 'currencies': 4, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 3, 'hold': 2, 'swiss': 4, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 42, 'u': 71, 'l': 84, 'trims': 4, 'early': 1, 'gains': 12, 'back': 7, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 2, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 3, 'forex': 94, 'edges': 4, 'higher': 10, 'amid': 3, 'lull': 2, 'trade': 11, 'rhetoric': 2, 'price': 1, 'action': 1, 'setups': 2, 'sets': 1

{'usd': 22, 'chf': 19, 'retreats': 2, '0': 10, '99': 2, 'trump': 1, 'comments': 2, 'weigh': 2, 'market': 2, 'sentiment': 2, 'remains': 20, 'â': 6, 'highly': 1, 'valuedâ': 1, 'danske': 1, 'bank': 2, 'technical': 5, 'analysis': 4, 'bearish': 1, 'double': 1, 'top': 2, 'formation': 1, 'h4': 1, 'positive': 2, '9875': 1, '72': 1, 'commerzbank': 3, 'plummets': 1, 'us': 4, 'dollar': 103, 'index': 15, 'extends': 5, 'losses': 6, 'safe': 3, 'currencies': 4, 'continue': 1, 'weaken': 1, 'risk': 2, 'onâ': 1, 'mood': 1, 'persists': 1, 'snb': 1, 'leaves': 1, 'key': 2, 'rate': 3, 'hold': 2, 'swiss': 4, 'national': 1, 'keeps': 2, '75': 1, 'widely': 1, 'expected': 1, 'n': 42, 'u': 74, 'l': 84, 'trims': 4, 'early': 1, 'gains': 12, 'back': 7, 'around': 2, 'mid': 3, '9900s': 1, 'demand': 2, 'havens': 1, 'weakens': 1, 'improves': 1, 'looks': 2, 'consolidative': 1, 'time': 3, 'forex': 100, 'edges': 4, 'higher': 10, 'amid': 3, 'lull': 2, 'trade': 14, 'rhetoric': 2, 'price': 1, 'action': 1, 'setups': 2, 'sets':

In [11]:
N = len(df_headlines)
average_word_count = df_frequency[1:2].sum(axis=1)/N
word_occurrence = [0 for x in range(df_frequency.count(axis='columns')[1])]
headlines_list = df_headlines['title'].tolist()
word_list = df_frequency[0:1].values.tolist()[0]

for word_index, word in enumerate(word_list):
    for headline in headlines_list:
        print(word)
        hd = RegexpTokenizer(r'\w+').tokenize(headline)
        print(hd)
        if word.upper() in hd:
            word_occurrence[word_index] += 1

# Add the number of occurrences to the data frame
df_frequency = df_frequency.append([word_occurrence], ignore_index=True)
df_frequency = df_frequency.T
df_frequency.columns = ["word", "frequency", "dfi"]

# Calculate the weight
word_weight = [0 for x in range(df_frequency.count(axis='columns')[1])]
# endregion

usd
['USD', 'CHF', 'retreats', 'below', '0', '99', 'as', 'Trump', 's', 'comments', 'weigh', 'on', 'market', 'sentiment']
usd
['CHF', 'remains', 'â', 'highly', 'valuedâ', 'â', 'Danske', 'Bank']
usd
['USD', 'CHF', 'Technical', 'Analysis', 'a', 'bearish', 'double', 'top', 'formation', 'on', 'H4']
usd
['USD', 'CHF', 'positive', 'while', 'above', '0', '9875', '72', 'â', 'Commerzbank']
usd
['USD', 'CHF', 'plummets', 'to', '0', '99', 'as', 'US', 'Dollar', 'Index', 'extends', 'losses']
usd
['Safe', 'Haven', 'Currencies', 'Continue', 'to', 'Weaken', 'as', 'â', 'Risk', 'Onâ', 'Mood', 'Persists']
usd
['SNB', 'Leaves', 'Key', 'Rate', 'On', 'Hold']
usd
['Swiss', 'National', 'Bank', 'keeps', 'key', 'rate', 'on', 'hold', 'at', '0', '75', 'as', 'widely', 'expected']
usd


TypeError: expected string or bytes-like object

In [10]:
word_list

['usd',
 'chf',
 'retreats',
 '0',
 '99',
 'trump',
 'comments',
 'weigh',
 'market',
 'sentiment',
 'remains',
 'â',
 'highly',
 'valuedâ',
 'danske',
 'bank',
 'technical',
 'analysis',
 'bearish',
 'double',
 'top',
 'formation',
 'h4',
 'positive',
 '9875',
 '72',
 'commerzbank',
 'plummets',
 'us',
 'dollar',
 'index',
 'extends',
 'losses',
 'safe',
 'currencies',
 'continue',
 'weaken',
 'risk',
 'onâ',
 'mood',
 'persists',
 'snb',
 'leaves',
 'key',
 'rate',
 'hold',
 'swiss',
 'national',
 'keeps',
 '75',
 'widely',
 'expected',
 'n',
 'u',
 'l',
 'trims',
 'early',
 'gains',
 'back',
 'around',
 'mid',
 '9900s',
 'demand',
 'havens',
 'weakens',
 'improves',
 'looks',
 'consolidative',
 'time',
 'forex',
 'edges',
 'higher',
 'amid',
 'lull',
 'trade',
 'rhetoric',
 'price',
 'action',
 'setups',
 'sets',
 'fresh',
 '11',
 'month',
 'high',
 'gold',
 'prices',
 'struggle',
 'find',
 'footing',
 'rallies',
 'loses',
 'bullish',
 'momentum',
 'dxy',
 'slips',
 'mixed',
 'data'

In [12]:
df_headlines

,Unnamed: 0,date,source,title,url,lexicon_sent,market_sent,aaii_sent,final_sent,sent_word
0,0,2018-06-22,FXStreet,USD/CHF retreats below 0.99 as Trump's comment...,https://invst.ly/7tjvs,0.000000,NaN,NaN,0.000000,neutral
1,1,2018-06-22,FXStreet,CHF remains âhighly valuedâ â Danske Bank,https://invst.ly/7tglx,0.000000,NaN,NaN,0.000000,neutral
2,2,2018-06-22,FXStreet,USD/CHF Technical Analysis: a bearish double-t...,https://invst.ly/7tezn,0.000000,NaN,NaN,0.000000,neutral
3,3,2018-06-22,FXStreet,USD/CHF positive while above 0.9875/72 â Com...,https://invst.ly/7tdxa,0.000000,NaN,NaN,0.000000,neutral
4,4,2018-06-21,FXStreet,USD/CHF plummets to 0.99 as US Dollar Index ex...,https://invst.ly/7t5ye,-0.509792,NaN,NaN,-0.509792,neg
5,5,2018-06-21,Dailyfx,Safe Haven Currencies Continue to Weaken as â...,https://invst.ly/7s-v7,-1.493064,NaN,NaN,-1.493064,neg
6,6,2018-06-21,Investing.com,SNB Leaves Key Rate On Hold,https://investing.com/news/economic-indicators...,0.000000,NaN,NaN,0.000000,neutral
7,7,2018-06-21,FXStreet,Swiss National Bank keeps key rate on-hold at ...,https://invst.ly/7szeg,0.000000,NaN,NaN,0.000000,neutral
8,8,NaT,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,neutral
9,9,NaT,NaN,NaN,NaN,-0.746532,NaN,NaN,-0.746532,neg
